In [2]:
#Project imports
import os
import json
import time
import math
import torch
import numpy as np
import shutil
import torch.nn as nn, optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torch.autograd import Variable
from datetime import datetime
from collections import OrderedDict
from utils.target_transforms import ClassLabel, VideoID
from utils.temporal_transforms import TemporalBeginCrop, TemporalCenterCrop, TemporalRandomCrop
from utils.spatial_transforms import Normalize, Scale, CenterCrop, CornerCrop, Compose, ToTensor, TemporalRandomCrop, MultiScaleCornerCrop, MultiScaleRandomCrop, RandomHorizontalFlip
from dataset.jester import Jester
from utils.train_utils import Logger


ModuleNotFoundError: No module named 'torch'

In [ ]:
# Configuration setting for the entire notebook will be handled here
class Path:
    """
    Project constant path class
    -----
    Attributes:

    - dataset_path (str): Path to dataset 

    - annotation_path (str): Path to dataset annotation {validation and training list labels}

    - result_path (str): Store model results and weights to be dumped in this folder
    """
    
    dataset_path = 'dataset/jester'

    annotation_path = 'dataset/annotation/jester.json'

    result_path = 'results'

class Config:
    """
    Model training Configuration class
    -----

    Attributes:

    - arch: Name of architecture being used
    - learning_rate (float) - how many filters to add each layer (k in paper)
    - n_epoch (int) - how many layers in each pooling block
    - lr_patience:
    - momentum:
    - begin_epoch:
    - n_epochs:
    - lr_steps:
    - weight_decay:
    - nesterov:
    - dataset:
    - model:
    - width_mult:
    - batch_size:
    - n_threads:
    -  dampening:
    - modality:
    - sample_duration:
    - sample_size:
    - n_val_samples:
    - downsample:
    - optimiser:
    - cuda:
    - dataset_path:
    - annotation_path:
    - result_path:
    - n_classes:
    - initial_scale:
    - n_scales:
    - scale_step:
    - norm_value:
    - mean_dataset:
    - mean_norm:
    - std_norm: 

    Usage:
    ------

        from config import Config
        ``Config.learning_rate``
    """
  

    arch = 'Densenet'

    arch_type = 161

    learning_rate = 0.1

    lr_patience = 10

    momentum = 0.9

    begin_epoch = 1

    n_epochs = 100

    n_classes = 27

    lr_steps = [25, 40, 55, 70] # (lr_rate * (0.1 ** step_position ))  =>  (0.1 (0.1 ** 1))

    weight_decay = 1e-3

    nesterov = False

    dataset = 'jester'

    model = 'densenet3d'

    activation = 'relu' # options: 'relu' or 'leaky_relu'

    classifier = 'Linear' # options: 'Softmax' or 'Linear'

    negative_slope = 0.1
    
    train = True

    validation = True

    test = False

    seed = 1000

    width_mult = 1

    batch_size = 16

    n_threads = 16

    dampening = 0.9

    modality = 'RGB'

    sample_duration = 16

    sample_size = 112

    n_val_samples = 1

    downsample = 2

    optimizer = 'SGD' # options: 'Adam' or 'SGD'

    betas = (0.9, 0.999)

    eps = 1e-8

    amsgrad = False

    cuda = True

    dataset_path = Path.dataset_path

    annotation_path = Path.annotation_path

    result_path = Path.result_path

    resume_path = ''

    initial_scale = 1

    n_scales = 5

    scale_step = 0.84089641525

    norm_value = 255

    mean_dataset = 'activitynet'

    dataset = ''

    store_name = ''

    train_crop = 'random'

    mean_norm = False,

    std_norm = False,

    softmax_in_test = True

    scale_in_test = 1.0

    test_subset = 'val'

    crop_position_in_test = 'c'

In [ ]:
# Denset3D Model 
class _DenseLayer(nn.Sequential):
    
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
        super().__init__()
        self.add_module('norm1', nn.BatchNorm3d(num_input_features))
        self.add_module('relu1', set_activation())
        self.add_module(
            'conv1',
            nn.Conv3d(num_input_features,
                      bn_size * growth_rate,
                      kernel_size=1,
                      stride=1,
                      bias=False))
        self.add_module('norm2', nn.BatchNorm3d(bn_size * growth_rate))
        self.add_module('relu2', set_activation())
        self.add_module(
            'conv2',
            nn.Conv3d(bn_size * growth_rate,
                      growth_rate,
                      kernel_size=3,
                      stride=1,
                      padding=1,
                      bias=False))
        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = super().forward(x)
        if self.drop_rate > 0:
            new_features = F.dropout(new_features,
                                     p=self.drop_rate,
                                     training=self.training)
        return torch.cat([x, new_features], 1)


class _DenseBlock(nn.Sequential):

    def __init__(self, num_layers, num_input_features, bn_size, growth_rate,
                 drop_rate):
        super().__init__()
        for i in range(num_layers):
            layer = _DenseLayer(num_input_features + i * growth_rate,
                                growth_rate, bn_size, drop_rate)
            self.add_module('denselayer{}'.format(i + 1), layer)


class _Transition(nn.Sequential):

    def __init__(self, num_input_features, num_output_features):
        super().__init__()
        self.add_module('norm', nn.BatchNorm3d(num_input_features))
        self.add_module('relu', set_activation())
        self.add_module(
            'conv',
            nn.Conv3d(num_input_features,
                      num_output_features,
                      kernel_size=1,
                      stride=1,
                      bias=False))
        self.add_module('pool', nn.AvgPool3d(kernel_size=2, stride=2))


class DenseNet(nn.Module):
    """Densenet-BC model class
    Args:
        growth_rate (int) - how many filters to add each layer (k in paper)
        block_config (list of 4 ints) - how many layers in each pooling block
        num_init_features (int) - the number of filters to learn in the first convolution layer
        bn_size (int) - multiplicative factor for number of bottle neck layers
          (i.e. bn_size * k features in the bottleneck layer)
        drop_rate (float) - dropout rate after each dense layer
        num_classes (int) - number of classification classes
    """

    def __init__(self,
                 n_input_channels=3,
                 conv1_t_size=7,
                 conv1_t_stride=1,
                 no_max_pool=False,
                 growth_rate=32,
                 block_config=(6, 12, 24, 16),
                 num_init_features=64,
                 bn_size=4,
                 drop_rate=0,
                 num_classes=1000):
    
        super().__init__()

        # First convolution
        self.features = [('conv1',
                          nn.Conv3d(n_input_channels,
                                    num_init_features,
                                    kernel_size=(conv1_t_size, 7, 7),
                                    stride=(conv1_t_stride, 2, 2),
                                    padding=(conv1_t_size // 2, 3, 3),
                                    bias=False)),
                         ('norm1', nn.BatchNorm3d(num_init_features)),
                         ('relu1', set_activation())]
        if not no_max_pool:
            self.features.append(
                ('pool1', nn.MaxPool3d(kernel_size=3, stride=2, padding=1)))
        self.features = nn.Sequential(OrderedDict(self.features))

        # Each denseblock
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(num_layers=num_layers,
                                num_input_features=num_features,
                                bn_size=bn_size,
                                growth_rate=growth_rate,
                                drop_rate=drop_rate)
            self.features.add_module('denseblock{}'.format(i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _Transition(num_input_features=num_features,
                                    num_output_features=num_features // 2)
                self.features.add_module('transition{}'.format(i + 1), trans)
                num_features = num_features // 2

        # Final batch norm
        self.features.add_module('norm5', nn.BatchNorm3d(num_features))

        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                m.weight = nn.init.kaiming_normal_(m.weight, mode='fan_out')
            elif isinstance(m, nn.BatchNorm3d) or isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

        # Linear layer
        if Config.classifier == 'Softmax' or Config.classifier == 'LogSoftmax':
            self.classifier = nn.LogSoftmax()
        else:
            self.classifier = nn.Linear(num_features, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight,
                                        mode='fan_out',
                                        nonlinearity=Config.activation)
            elif isinstance(m, nn.BatchNorm3d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        features = self.features(x)
        activation_method = getattr(F, Config.activation)
        out = activation_method(features, inplace=True)
        out = F.adaptive_avg_pool3d(out,
                                    output_size=(1, 1,
                                                 1)).view(features.size(0), -1)
        out = self.classifier(out)
        return out


def getModel(**kwargs):
    if Config.arch_type==121:
        return DenseNet(num_init_features=64,
                         growth_rate=32,
                         block_config=(6, 12, 24, 16), **kwargs)
    elif Config.arch_type==161:
        return DenseNet(num_init_features=96,
                         growth_rate=48,
                         block_config=(6, 12, 36, 24), **kwargs)
    elif Config.arch_type==169:
        return DenseNet(num_init_features=64,
                         growth_rate=32,
                         block_config=(6, 12, 32, 32), **kwargs)
    elif Config.arch_type==201:
        return DenseNet(num_init_features=64,
                         growth_rate=32,
                         block_config=(6, 12, 48, 32), **kwargs)


def set_activation():
    assert(Config.activation =='leaky_relu' or Config.activation == 'relu')
    if Config.activation == 'leaky_relu':
        return nn.LeakyReLU(Config.negative_slope, inplace=True)
    return nn.ReLU(inplace=True)

In [ ]:
#Utility Functions
def get_mean(norm_value=255, dataset='activitynet'):
    assert dataset in ['activitynet', 'kinetics']

    if dataset == 'activitynet':
        return [
            114.7748 / norm_value, 107.7354 / norm_value, 99.4750 / norm_value
        ]
    elif dataset == 'kinetics':
        # Kinetics (10 videos for each class)
        return [
            110.63666788 / norm_value, 103.16065604 / norm_value,
            96.29023126 / norm_value
        ]

def get_std(norm_value=255):
        # Kinetics (10 videos for each class)
    return [
        38.7568578 / norm_value, 37.88248729 / norm_value,
        40.02898126 / norm_value
    ]

def set_norm_method(mean, std):
    """
    Set normalization method based on mean and std

    Parameters:
    ------------
    mean : float 
        Mean value for dataset
    std : float 
        Standard deviation for dataset
    
    Returns
    ---------        
    Normalize
        Normalise class

    """
    if Config.mean_norm and not Config.std_norm:
        return Normalize([0, 0, 0], [1, 1, 1])
    elif not Config.std_norm:
        return Normalize(mean, [1, 1, 1])
    else: 
        return Normalize(mean, std) 

def set_optimizer(model):
    """
    Set optimizer algorithm:

    Args:
    ----
    model: torch.nn.Module
        Instance of nn.Module
    
    Returns: 
    --------
    torch.optim.Optimizer
    """
    assert(Config.optimizer == 'SGD' or Config.optimizer == 'Adam')

    if Config.optimizer == 'SGD':
        if Config.nesterov:
            dampening = 0
        else:
            dampening = Config.dampening

        return optim.SGD(model.parameters(), 
            lr=Config.learning_rate, 
            momentum=Config.momentum,
            dampening=dampening,
            weight_decay=Config.weight_decay,
            nesterov=Config.nesterov)

    return optim.Adam(model.parameters(), 
        lr=Config.learning_rate, 
        betas=Config.betas,
        eps=Config.eps,
        weight_decay=Config.weight_decay,
        amsgrad=Config.amsgrad)

def set_crop_method(scales):
    """
    Set cropping method to be applied on the dataset in memory

    Args:
    -----
    scales : list 
        list of scaling ratios

    Returns:
    ---------
    Spatial Transformation

    """
    assert Config.train_crop in ['random','corner', 'center']
    crop = Config.train_crop
    if crop == 'random':
        return MultiScaleRandomCrop(scales, Config.sample_size)
    elif crop == 'corner':
        return MultiScaleCornerCrop(scales, Config.sample_size)
    elif crop == 'center':
        return MultiScaleCornerCrop(scales, Config.sample_size, crop_positions=['c'])


def get_training_set(spatial_transform, temporal_transform,
                     target_transform):
    return Jester(
        Config.dataset_path,
        Config.annotation_path,
        'training',
        spatial_transform=spatial_transform,
        temporal_transform=temporal_transform,
        target_transform=target_transform,
        sample_duration=Config.sample_duration)

def get_validation_set(spatial_transform, temporal_transform,
                       target_transform):
        return Jester(
            Config.dataset_path,
            Config.annotation_path,
            'validation',
            Config.n_val_samples,
            spatial_transform,
            temporal_transform,
            target_transform,
            sample_duration=Config.sample_duration)

def save_checkpoint(state, is_best, store_name):
    torch.save(state, '%s/%s_checkpoint.pth' % (Config.result_path, store_name))
    if is_best:
        shutil.copyfile('%s/%s_checkpoint.pth' % (Config.result_path, store_name),'%s/%s_best.pth' % (Config.result_path, Config.store_name))


def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr_new = Config.learning_rate * (0.1 ** (sum(epoch >= np.array(Config.lr_steps))))
    for param_group in optimizer.param_groups:
          param_group['lr'] = lr_new
        #param_group['lr'] = opt.learning_rate



In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def calculate_accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res
    
def train_epoch(epoch, data_loader, model, criterion, optimizer,
                epoch_logger, batch_logger):
    print('Training at epoch {}/{}'.format(epoch, Config.n_epochs))

    model.train()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    end_time = time.time()
    for i, (inputs, targets) in enumerate(data_loader):
        data_time.update(time.time() - end_time)

        if Config.cuda:
            targets = targets.cuda()
        inputs = Variable(inputs)
        targets = Variable(targets)
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        losses.update(loss.data, inputs.size(0))
        prec1, prec5 = calculate_accuracy(outputs.data, targets.data, topk=(1,5))
        top1.update(prec1, inputs.size(0))
        top5.update(prec5, inputs.size(0))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_time.update(time.time() - end_time)
        end_time = time.time()

        batch_logger.log({
            'epoch': epoch,
            'batch': i + 1,
            'iter': (epoch - 1) * len(data_loader) + (i + 1),
            'loss': losses.val.item(),
            'prec1': top1.val.item(),
            'prec5': top5.val.item(),
            'lr': optimizer.param_groups[0]['lr']
        })
        if i % 10 == 0:
            print('Epoch: [{0}][{1}/{2}]\t lr: {lr:.5f}\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.5f} ({top1.avg:.5f})\t'
                  'Prec@5 {top5.val:.5f} ({top5.avg:.5f})'.format(
                      epoch,
                      i,
                      len(data_loader),
                      batch_time=batch_time,
                      data_time=data_time,
                      loss=losses,
                      top1=top1,
                      top5=top5,
                      lr=optimizer.param_groups[0]['lr']))

    epoch_logger.log({
        'epoch': epoch,
        'loss': losses.avg.item(),
        'prec1': top1.avg.item(),
        'prec5': top5.avg.item(),
        'lr': optimizer.param_groups[0]['lr']
    })

    #if epoch % opt.checkpoint == 0:
    #    save_file_path = os.path.join(opt.result_path,
    #                                  'save_{}.pth'.format(epoch))
    #    states = {
    #        'epoch': epoch + 1,
    #        'arch': opt.arch,
    #        'state_dict': model.state_dict(),
    #        'optimizer': optimizer.state_dict(),
    #    }
    #    torch.save(states, save_file_path)


# TODO: Try using CTross validation.

def val_epoch(epoch, data_loader, model, criterion, logger):
    print('validation at epoch {}'.format(epoch))

    model.eval()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    end_time = time.time()
    for i, (inputs, targets) in enumerate(data_loader):
        data_time.update(time.time() - end_time)

        if Config.cuda:
            targets = targets.cuda()
        with torch.no_grad():
            inputs = Variable(inputs)
            targets = Variable(targets)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        prec1, prec5 = calculate_accuracy(outputs.data, targets.data, topk=(1,5))
        top1.update(prec1, inputs.size(0))
        top5.update(prec5, inputs.size(0))

        losses.update(loss.data, inputs.size(0))

        batch_time.update(time.time() - end_time)
        end_time = time.time()

        print('Epoch: [{0}][{1}/{2}]\t'
              'Time {batch_time.val:.5f} ({batch_time.avg:.5f})\t'
              'Data {data_time.val:.5f} ({data_time.avg:.5f})\t'
              'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
              'Prec@1 {top1.val:.5f} ({top1.avg:.5f})\t'
              'Prec@5 {top5.val:.5f} ({top5.avg:.5f})'.format(
                  epoch,
                  i + 1,
                  len(data_loader),
                  batch_time=batch_time,
                  data_time=data_time,
                  loss=losses,
                  top1=top1,
                  top5=top5))

    logger.log({'epoch': epoch,
                'loss': losses.avg.item(),
                'prec1': top1.avg.item(),
                'prec5': top5.avg.item()})

    return losses.avg.item(), top1.avg.item()


def calculate_video_results(output_buffer, video_id, test_results, class_names):
    video_outputs = torch.stack(output_buffer)
    average_scores = torch.mean(video_outputs, dim=0)
    sorted_scores, locs = torch.topk(average_scores, k=10)

    video_results = []
    for i in range(sorted_scores.size(0)):
        video_results.append({
            'label': class_names[int(locs[i])],
            'score': float(sorted_scores[i])
        })

    test_results['results'][video_id] = video_results


def test(data_loader, model, class_names):

    model.eval()

    batch_time = AverageMeter()
    data_time = AverageMeter()

    end_time = time.time()
    output_buffer = []
    previous_video_id = ''
    test_results = {'results': {}}
    for i, (inputs, targets) in enumerate(data_loader):
        data_time.update(time.time() - end_time)

        with torch.no_grad():
            inputs = Variable(inputs)
        outputs = model(inputs)
        if Config.softmax_in_test:
            outputs = F.softmax(outputs, dim=1)

        for j in range(outputs.size(0)):
            if not (i == 0 and j == 0) and targets[j] != previous_video_id:
                calculate_video_results(output_buffer, previous_video_id,
                                        test_results, class_names)
                output_buffer = []
            output_buffer.append(outputs[j].data.cpu())
            previous_video_id = targets[j]

        if (i % 100) == 0:
            with open(
                    os.path.join(Config.result_path, '{}.json'.format(
                        Config.test_subset)), 'w') as f:
                json.dump(test_results, f)

        batch_time.update(time.time() - end_time)
        end_time = time.time()

        print('[{}/{}]\t'
              'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
              'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'.format(
                  i + 1,
                  len(data_loader),
                  batch_time=batch_time,
                  data_time=data_time))
    with open(
            os.path.join(Config.result_path, '{}.json'.format(Config.test_subset)),
            'w') as f:
        json.dump(test_results, f)

def evaluate_model(model):
    torch.manual_seed(Config.seed)
    model = getModel(Config.n_classes).cuda()
    model = nn.DataParallel(model, device_ids=None)
    params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(params);
    mean = get_mean(Config.norm_value, dataset = Config.mean_dataset)
    std = get_std(Config.norm_value)
    norm_method = set_norm_method(mean, std)

    spatial_transform = Compose([
        Scale(int(Config.sample_size / Config.scale_in_test)),
        CornerCrop(Config.sample_size, Config.crop_position_in_test),
        ToTensor(Config.norm_value), norm_method
    ])

    temporal_transform = TemporalRandomCrop(Config.sample_duration, Config.downsample)
    
    target_transform = ClassLabel()

    # test_data = get_test_set(spatial_transform, temporal_transform, target_transform)
    test_loader = torch.utils.data.DataLoader(
            test_data,
            batch_size=1,
            shuffle=False,
            num_workers=Config.n_threads,
            pin_memory=True)
    model.eval()

    recorder = []

    batch_time = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    start = time.time()

    for i, (inputs, targets) in enumerate(test_loader):
        if Config.cuda:
            targets = targets.cuda(async=True)
        #inputs = Variable(torch.squeeze(inputs), volatile=True)
        inputs = Variable(inputs, volatile=True)
        targets = Variable(targets, volatile=True)
        outputs = model(inputs)

        recorder.append(outputs.data.cpu().numpy().copy())
        #outputs = torch.unsqueeze(torch.mean(outputs, 0), 0)
        prec1, prec5 = calculate_accuracy(outputs, targets, topk=(1, 5))

        top1.update(prec1, inputs.size(0))
        top5.update(prec5, inputs.size(0))

        batch_time.update(time.time() - start)
        end_time = time.time()

        print('[{0}/{1}]\t'
            'Time {batch_time.val:.5f} ({batch_time.avg:.5f})\t'
            'prec@1 {top1.avg:.5f} prec@5 {top5.avg:.5f}'.format(
                i + 1,
                len(test_loader),
                batch_time=batch_time,
                top1=top1,
                top5=top5))

    video_pred = [np.argmax(np.mean(x, axis=0)) for x in recorder]
    print(video_pred)

    with open('dataset/annotation/categories.txt') as f:
        lines = f.readlines()
        categories = [item.rstrip() for item in lines]

    name_list = [x.strip().split()[0] for x in open('dataset/annotation/vallist.txt')]

    order_dict = {e:i for i, e in enumerate(sorted(name_list))}
    reorder_output = [None] * len(recorder)
    reorder_pred = [None] * len(recorder)
    output_csv = []

    for i in range(len(recorder)):
        idx = order_dict[name_list[i]]
        reorder_output[idx] = recorder[i]
        reorder_pred[idx] = video_pred[i]
        output_csv.append('%s;%s'%(name_list[i],
                                categories[video_pred[i]]))

        with open(Config.result_path +'/'+Config.dataset + '_predictions.csv','w') as f:
            f.write('\n'.join(output_csv))
 
    print('-----Evaluation is finished------')
    print('Overall Prec@1 {:.05f}% Prec@5 {:.05f}%'.format(top1.avg, top5.avg))
    

In [ ]:
# Main Code execution begin here
start =  datetime.now()

print(torch.cuda.is_available())

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
    
print(f'Graphic Cart Used for the experiment: {device}')

# Initialize model
model = getModel(Config.n_classes).cuda()
model = nn.DataParallel(model, device_ids=None)
params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(params);

criterion = nn.CrossEntropyLoss().cuda()

In [ ]:
mean = get_mean(Config.norm_value, dataset = Config.mean_dataset) 
std = get_std(Config.norm_value)
store_name = '_'.join([Config.dataset, Config.model, str(Config.width_mult) + 'x',
                               Config.modality, str(Config.sample_duration)])

norm_method = set_norm_method(mean, std)


In [ ]:
 # set scaling values
scales = [Config.initial_scale]
for i in range(1, Config.n_scales):
    scales.append(scales[-1] * Config.scale_step)

optimizer = set_optimizer(model)


if Config.train:
    crop_method = set_crop_method(scales)

    spatial_transform = Compose([
        RandomHorizontalFlip(),
        #RandomRotate(),
        #RandomResize(),
        crop_method,
        #MultiplyValues(),
        #Dropout(),
        #SaltImage(),
        #Gaussian_blur(),
        #SpatialElasticDisplacement(),
        ToTensor(Config.norm_value), norm_method
    ])
    temporal_transform = TemporalRandomCrop(Config.sample_duration, Config.downsample)
    target_transform = ClassLabel()
    training_data = get_training_set(spatial_transform,temporal_transform,target_transform)

    train_loader = torch.utils.data.DataLoader(
            training_data,
            batch_size=Config.batch_size,
            shuffle=True,
            num_workers=Config.n_threads,
            pin_memory=True)

    train_logger = Logger(
            os.path.join(Config.result_path, 'DenseNet'+ str(Config.arch_type) +'train.log'),
            ['epoch', 'loss', 'prec1', 'prec5', 'lr'])
            
    train_batch_logger = Logger(
        os.path.join(Config.result_path, 'DenseNet'+ str(Config.arch_type) + 'train_batch.log'),
        ['epoch', 'batch', 'iter', 'loss', 'prec1', 'prec5', 'lr'])

    # Set optimizer algorithm

    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min', 
        patience=Config.lr_patience)

# If set validation set. Apply spatial transformations. 
if Config.validation:
    spatial_transform = Compose([
        Scale(Config.sample_size),
        CenterCrop(Config.sample_size),
        ToTensor(Config.norm_value), norm_method
    ])
    #temporal_transform = LoopPadding(opt.sample_duration)
    temporal_transform = TemporalCenterCrop(Config.sample_duration, Config.downsample)
    target_transform = ClassLabel()
    validation_data = get_validation_set(spatial_transform, temporal_transform, target_transform)
    val_loader = torch.utils.data.DataLoader(
        validation_data,
        batch_size=Config.batch_size,
        shuffle=False,
        num_workers=Config.n_threads,
        pin_memory=True)
    val_logger = Logger(
        os.path.join(Config.result_path, 'DenseNet'+ str(Config.arch_type) + 'val.log'), ['epoch', 'loss', 'prec1', 'prec5'])

best_prec1 = 0
if Config.resume_path:
    print('loading checkpoint {}'.format(Config.resume_path))
    checkpoint = torch.load(Config.resume_path)
    assert Config.arch == checkpoint['arch']
    best_prec1 = checkpoint['best_prec1']
    Config.begin_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])

for i in range(Config.begin_epoch, Config.n_epochs + 1):
    if Config.train:
        adjust_learning_rate(optimizer, i)
        
        train_epoch(i, train_loader, model, criterion, optimizer,
                        train_logger, train_batch_logger)
        state = {
            'epoch': i,
            'arch': Config.arch,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'best_prec1': best_prec1
            }
        save_checkpoint(state, False, store_name)

    if  Config.validation:
        validation_loss, prec1 = val_epoch(i, val_loader, model, criterion,
            val_logger)
                
        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        state = {
            'epoch': i,
            'arch': Config.arch,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'best_prec1': best_prec1
            }
        save_checkpoint(state, is_best, store_name)

if Config.test:
    # evaluate_model(densenet)
    spatial_transform = Compose([
        Scale(int(Config.sample_size / Config.scale_in_test)),
        CornerCrop(Config.sample_size, Config.crop_position_in_test),
        ToTensor(Config.norm_value), norm_method
    ])
    # temporal_transform = LoopPadding(opt.sample_duration, opt.downsample)
    temporal_transform = TemporalRandomCrop(Config.sample_duration, Config.downsample)
    target_transform = VideoID()

    test_data = get_test_set(spatial_transform, temporal_transform, target_transform)
    test_loader = torch.utils.data.DataLoader(
        test_data,
        batch_size=Config.batch_size,
        shuffle=False,
        num_workers=Config.n_threads,
        pin_memory=True)
    test(test_loader, model, test_data.class_names)

time_elapsed = datetime.now() - start 
print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))
    